In [1]:
import json
from urllib.parse import urlencode
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime

In [2]:
url = "https://www.bbc.com/weather/1275339"
response = requests.get(url)
response

<Response [200]>

In [3]:
required_city = "Mumbai"
location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
   'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
   's': required_city,
   'stack': 'aws',
   'locale': 'en',
   'filter': 'international',
   'place-types': 'settlement,airport,district',
   'order': 'importance',
   'a': 'true',
   'format': 'json'
})
location_url

'https://locator-service.api.bbci.co.uk/locations?api_key=AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv&s=Mumbai&stack=aws&locale=en&filter=international&place-types=settlement%2Cairport%2Cdistrict&order=importance&a=true&format=json'

In [47]:
result = requests.get(location_url).json()
print(result)

{'response': {'results': {'results': [{'id': '1275339', 'name': 'Mumbai', 'container': 'India', 'containerId': 1269750, 'language': 'en', 'timezone': 'Asia/Kolkata', 'country': 'IN', 'latitude': 19.07283, 'longitude': 72.88261, 'placeType': 'settlement'}], 'totalResults': 1}}}


In [48]:
url = 'https://www.bbc.com/weather/'+result['response']['results']['results'][0]['id']
print(url)

https://www.bbc.com/weather/1275339


In [49]:
soup = BeautifulSoup(response.content, 'lxml')
soup

<!DOCTYPE html>
<html class="b-reith-sans-font b-pw-1280 no-touch wr-core wr-svg no-js" data-location-id="1275339" data-location-name="Mumbai" data-wr-unit--temperature="c" data-wr-unit--windspeed="mph" id="weather-forecast" lang="en">
<head>
<meta content="width=device-width, initial-scale=1, user-scalable=1" name="viewport"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="on" http-equiv="cleartype"/>
<link href="https://ssl.bbc.co.uk/" rel="dns-prefetch"/>
<link href="http://sa.bbc.co.uk/" rel="dns-prefetch"/>
<link href="http://ichef-1.bbci.co.uk/" rel="dns-prefetch"/>
<link href="http://ichef.bbci.co.uk/" rel="dns-prefetch"/>
<style>
        [data-wr-unit--temperature="c"] .wr-c-map__temperature-f,
        [data-wr-unit--temperature="f"] .wr-c-map__temperature-c,
        [data-wr-unit--windspeed="mph"] .wr-c-map__wind-kph,
        [data-wr-unit--windspeed="kph"] .wr-c-map__wind-mph {
            position: absolute;
           

In [50]:
daily_high_values = soup.find_all('span', class_ = 'wr-day-temperature__high-value')
daily_high_values

[<span class="wr-day-temperature__high-value"><span class="wr-value--temperature"><span class="wr-value--temperature--c">34°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">93°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature"><span class="wr-value--temperature--c">34°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">93°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature"><span class="wr-value--temperature--c">33°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">91°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature"><span class="wr-value--temperature--c">32°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">90°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature"><span class="w

In [51]:
daily_summary = soup.find('div', attrs={'class' : 'wr-day-summary'}).text
daily_summary

'A clear sky and light windsSunny and a gentle breezeSunny and a gentle breezeSunny and a gentle breezeSunny and a gentle breezeSunny and a gentle breezeSunny intervals and a gentle breezeSunny and a gentle breezeSunny and a moderate breezeSunny and a moderate breezeSunny and a gentle breezeSunny and a gentle breezeSunny and a gentle breezeSunny and a gentle breeze'

In [64]:
high=[]
low=[]
for high_value in daily_high_values:
    temp = high_value.text.split()
    high.append(temp[1])
    low.append(temp[0])
print(low,high)

['34°', '34°', '33°', '32°', '31°', '30°', '30°', '30°', '31°', '31°', '32°', '32°', '33°'] ['93°', '93°', '91°', '90°', '88°', '86°', '85°', '86°', '87°', '88°', '89°', '90°', '91°']


In [65]:
daily_summary_list = re.findall('[a-zA-Z][^A-Z]+', daily_summary)
daily_summary_list

['A clear sky and light winds',
 'Sunny and a gentle breeze',
 'Sunny and a gentle breeze',
 'Sunny and a gentle breeze',
 'Sunny and a gentle breeze',
 'Sunny and a gentle breeze',
 'Sunny intervals and a gentle breeze',
 'Sunny and a gentle breeze',
 'Sunny and a moderate breeze',
 'Sunny and a moderate breeze',
 'Sunny and a gentle breeze',
 'Sunny and a gentle breeze',
 'Sunny and a gentle breeze',
 'Sunny and a gentle breeze']

In [66]:
date_list = pd.date_range(datetime.today(), periods=len(daily_high_values)).tolist()
date_list

[Timestamp('2024-02-16 00:30:02.196447'),
 Timestamp('2024-02-17 00:30:02.196447'),
 Timestamp('2024-02-18 00:30:02.196447'),
 Timestamp('2024-02-19 00:30:02.196447'),
 Timestamp('2024-02-20 00:30:02.196447'),
 Timestamp('2024-02-21 00:30:02.196447'),
 Timestamp('2024-02-22 00:30:02.196447'),
 Timestamp('2024-02-23 00:30:02.196447'),
 Timestamp('2024-02-24 00:30:02.196447'),
 Timestamp('2024-02-25 00:30:02.196447'),
 Timestamp('2024-02-26 00:30:02.196447'),
 Timestamp('2024-02-27 00:30:02.196447'),
 Timestamp('2024-02-28 00:30:02.196447')]

In [67]:
date_list = [date_list[i].date().strftime('%y-%m-%d') for i in range(len(date_list))]
date_list

['24-02-16',
 '24-02-17',
 '24-02-18',
 '24-02-19',
 '24-02-20',
 '24-02-21',
 '24-02-22',
 '24-02-23',
 '24-02-24',
 '24-02-25',
 '24-02-26',
 '24-02-27',
 '24-02-28']

In [73]:
zipped = zip(date_list, high, low, daily_summary_list)
zipped

In [74]:
df = pd.DataFrame(list(zipped), columns=['Date', 'High', 'Low', 'Summary'])

In [75]:
display(df)

,Date,High,Low,Summary
0,24-02-16,93°,34°,A clear sky and light winds
1,24-02-17,93°,34°,Sunny and a gentle breeze
2,24-02-18,91°,33°,Sunny and a gentle breeze
3,24-02-19,90°,32°,Sunny and a gentle breeze
4,24-02-20,88°,31°,Sunny and a gentle breeze
5,24-02-21,86°,30°,Sunny and a gentle breeze
6,24-02-22,85°,30°,Sunny intervals and a gentle breeze
7,24-02-23,86°,30°,Sunny and a gentle breeze
8,24-02-24,87°,31°,Sunny and a moderate breeze
9,24-02-25,88°,31°,Sunny and a moderate breeze


In [84]:
location = soup.find('h1', class_='wr-c-location__name gel-paragon').text
location.split()

['Mumbai', '-', 'Weather', 'warnings', 'issued']

In [85]:
mumbai_csv = location.split()[0]+'.csv'
df.to_csv(mumbai_csv, index=None)

In [87]:
mumbai_xlsx = location.split()[0]+'.xlsx'
df.to_excel(mumbai_xlsx)